In [86]:
!pip install opendatasets

In [87]:
import opendatasets as od
import cv2
import numpy as np

od.download(
    "https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data"
)

Skipping, found downloaded files in "./store-sales-time-series-forecasting" (use force=True to force download)


In [88]:
import pandas as pd
train=pd.read_csv("/content/store-sales-time-series-forecasting/train.csv")
test=pd.read_csv("/content/store-sales-time-series-forecasting/test.csv")
stores=pd.read_csv("/content/store-sales-time-series-forecasting/stores.csv")
oil=pd.read_csv("/content/store-sales-time-series-forecasting/oil.csv")
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [89]:
train=pd.merge(train,stores,how="inner",on="store_nbr",suffixes=(None, ''))
test=pd.merge(test,stores,how="inner",on="store_nbr",suffixes=(None, ''))
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [90]:
train=pd.merge(train,oil,how="left",on="date",suffixes=(None, ''))
test=pd.merge(test,oil,how="left",on="date",suffixes=(None, ''))
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN


In [91]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   city         object 
 7   state        object 
 8   type         object 
 9   cluster      int64  
 10  dcoilwtico   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 274.7+ MB


In [92]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
train['date']=oe.fit_transform(train[['date']])
train['family']=oe.fit_transform(train[['family']])
train['city']=oe.fit_transform(train[['city']])
train['state']=oe.fit_transform(train[['state']])
train['type']=oe.fit_transform(train[['type']])
test['date']=oe.fit_transform(test[['date']])
test['family']=oe.fit_transform(test[['family']])
test['city']=oe.fit_transform(test[['city']])
test['state']=oe.fit_transform(test[['state']])
test['type']=oe.fit_transform(test[['type']])

In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         float64
 2   store_nbr    int64  
 3   family       float64
 4   sales        float64
 5   onpromotion  int64  
 6   city         float64
 7   state        float64
 8   type         float64
 9   cluster      int64  
 10  dcoilwtico   float64
dtypes: float64(7), int64(4)
memory usage: 274.7 MB


In [94]:
train.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
sales               0
onpromotion         0
city                0
state               0
type                0
cluster             0
dcoilwtico     928422
dtype: int64

In [95]:
test.isnull().sum()

id                0
date              0
store_nbr         0
family            0
onpromotion       0
city              0
state             0
type              0
cluster           0
dcoilwtico     7128
dtype: int64

In [96]:
from sklearn.impute import SimpleImputer
mssi=SimpleImputer(strategy="median")
train['dcoilwtico']=mssi.fit_transform(train[['dcoilwtico']])
test['dcoilwtico']=mssi.fit_transform(test[['dcoilwtico']])
train=pd.concat([train,train])

In [97]:
from sklearn.model_selection import train_test_split
train=train.drop(['id','date'],axis=1);
x_train,x_test,y_train,y_test=train_test_split(train.drop(['sales'],axis=1),train['sales'],test_size=0.2,random_state=0)

In [98]:
import xgboost as xgb
t=xgb.DMatrix(x_train,label=y_train)
params = {
    'objective': 'reg:squaredlogerror',
    'eval_metric': 'auc',
    'eta': 0.4,
    'max_depth': 2,
}
model=xgb.train(params,t,num_boost_round=150)
te=xgb.DMatrix(x_test)
y_pred=model.predict(te)

In [99]:
from sklearn.metrics import r2_score
r2_score(y_pred,y_test)

-1.749315181961408

In [100]:
id=test['id']
test=test.drop(['id','date'],axis=1)
te=xgb.DMatrix(test)
ans=model.predict(te)
answer=pd.DataFrame({
    'id':id,
    'sales':ans
})
answer.to_csv('./answer.csv',sep=',',index=False,header=True)